In [ ]:
import csv
import json
import pandas as pd
import numpy as np


In [ ]:
df=pd.read_xml('data-text.xml')
df.head()

,Name,Value,Display,Dataset,Attribute,Dimension,Observation
0,format,xml,None,NaN,NaN,NaN,NaN
1,baseurl,http://apps.who.int/gho/athena/data/,None,NaN,NaN,NaN,NaN
2,filter,COUNTRY:*;REGION:AFR;REGION:AMR;REGION:SEAR;RE...,None,NaN,NaN,NaN,NaN
3,asof,2014-10-22 15:12:28.0,None,NaN,NaN,NaN,NaN
4,metadata,active,None,NaN,NaN,NaN,NaN


In [ ]:
df.isnull().sum()

Name            4
Value           4
Display         8
Dataset        10
Attribute      10
Dimension      10
Observation    10
dtype: int64

In [ ]:
!pip install bs4

  Preparing metadata (setup.py) ... done
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1256 sha256=0c22aa19b12dd7218b40b9a2c9fb6c02ffe17b1c30149c01c7520fa2bc28e0a8
  Stored in directory: /root/.cache/pip/wheels/25/42/45/b773edc52acb16cd2db4cf1a0b47117e2f69bb4eb300ed0e70
Successfully built bs4


In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
with open('data-text.xml', 'r') as f:
    file = f.read()
soup = BeautifulSoup(file, 'xml')

In [ ]:
soup = BeautifulSoup(file, 'xml')
titles = soup.find_all('title')

In [ ]:
for data in titles:
    print(data.get_text())

In [ ]:
def extract_data(soup, label):
    dimension = soup.find('Dimension', {'Label': label})

    if dimension:
        return [code.Display.get_text() for code in dimension.find_all('Code') if code.Display]
    return []

In [ ]:
names=extract_data(soup, 'PUBLISHSTATE')

In [ ]:
for name in names:
    print(name)

Published


In [ ]:
dfc=pd.read_csv("data-text.csv")
dfc.head()

,Indicator,PUBLISH STATES,Year,WHO region,World Bank income group,Country,Sex,Display Value,Numeric,Low,High,Comments
0,Life expectancy at birth (years),Published,1990,Europe,High-income,Andorra,Both sexes,77,77.0,NaN,NaN,NaN
1,Life expectancy at birth (years),Published,2000,Europe,High-income,Andorra,Both sexes,80,80.0,NaN,NaN,NaN
2,Life expectancy at age 60 (years),Published,2012,Europe,High-income,Andorra,Female,28,28.0,NaN,NaN,NaN
3,Life expectancy at age 60 (years),Published,2000,Europe,High-income,Andorra,Both sexes,23,23.0,NaN,NaN,NaN
4,Life expectancy at birth (years),Published,2012,Eastern Mediterranean,High-income,United Arab Emirates,Female,78,78.0,NaN,NaN,NaN


In [ ]:
dfc.isna().sum()

Indicator                     0
PUBLISH STATES                0
Year                          0
WHO region                    0
World Bank income group       0
Country                       0
Sex                           0
Display Value                 0
Numeric                       0
Low                        4656
High                       4656
Comments                   4656
dtype: int64

In [ ]:
dfc.drop(columns=['Low', 'High', 'Comments'], inplace=True)

In [ ]:
dfc.isnull().sum()

Indicator                  0
PUBLISH STATES             0
Year                       0
WHO region                 0
World Bank income group    0
Country                    0
Sex                        0
Display Value              0
Numeric                    0
dtype: int64

In [ ]:
def findOutliers(df):

   q1=df.quantile(0.25)

   q3=df.quantile(0.75)
   IQR=q3-q1
   outliers = df[((df<(q1-1.5*IQR)) | (df>(q3+1.5*IQR)))]

   return outliers
def removeOutliers(df):

   q1=df.quantile(0.25)

   q3=df.quantile(0.75)
   IQR=q3-q1

   df = df[~((df < (q1 - 1.5 * IQR)) |(df > (q3 + 1.5 * IQR)))]
   return df

numerical=dfc[["Year","Display Value" , "Numeric"]]
for col in numerical.columns:
  outliers = findOutliers(numerical[col])
  print(f"outliers in {col} before preprocessing= {len(outliers)}")




outliers in Year before preprocessing= 0
outliers in Display Value before preprocessing= 0
outliers in Numeric before preprocessing= 0


In [ ]:
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go



fig1 = px.bar(dfc, x='Indicator', y='Year', labels={'y': ' Year', 'x': 'Indicator'})
fig2 = px.bar(dfc, x='Indicator', y='Display Value', labels={'y': 'Display Value', 'x': 'Indicator'})
fig3 = px.scatter(dfc, x='Indicator', y='Numeric', labels={'y': 'Numeric', 'x': 'Indicator'})

fig1.show()
fig2.show()
fig3.show()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
y = dfc['Display Value']
X = dfc[["Numeric","Year"]]


X_train, X_test, y_train, y_test = train_test_split(X,y,train_size = 0.8)
LR = LogisticRegression()
LR.fit(X_train,y_train)

predicted = LR.predict(X_test)
cfm = confusion_matrix(y_test,predicted)
print(cfm)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
yn = numerical['Display Value']
Xn = numerical[["Numeric","Year"]]


X_trainn, X_testn, y_trainn, y_testn = train_test_split(Xn,yn,train_size = 0.8)
LRn = LogisticRegression()
LRn.fit(X_trainn,y_trainn)

predictedn = LRn.predict(X_testn)
cfmn = confusion_matrix(y_testn,predictedn)
print(cfmn)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [ ]:
from sklearn.metrics import accuracy_score

LR = LogisticRegression()
LR.fit(X_train, y_train)

predicted = LR.predict(X_test)
accuracy = accuracy_score(y_test, predicted)
print(accuracy)


0.09656652360515021


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



In [ ]:
from sklearn.metrics import accuracy_score

accuracyn = accuracy_score(y_testn, predictedn)
print(accuracyn)


0.11051502145922747


the cleaned model has a much higher accuracy value.


In [ ]:
jsonData=open('data-text.json').read()
data=json.loads(jsonData)
for i in data:
  print(i)

{'Indicator': 'Life expectancy at birth (years)', 'PUBLISH STATES': 'Published', 'Year': 1990, 'WHO region': 'Europe', 'World Bank income group': 'High-income', 'Country': 'Andorra', 'Sex': 'Both sexes', 'Display Value': 77, 'Numeric': 77.0, 'Low': '', 'High': '', 'Comments': ''}
{'Indicator': 'Life expectancy at birth (years)', 'PUBLISH STATES': 'Published', 'Year': 2000, 'WHO region': 'Europe', 'World Bank income group': 'High-income', 'Country': 'Andorra', 'Sex': 'Both sexes', 'Display Value': 80, 'Numeric': 80.0, 'Low': '', 'High': '', 'Comments': ''}
{'Indicator': 'Life expectancy at age 60 (years)', 'PUBLISH STATES': 'Published', 'Year': 2012, 'WHO region': 'Europe', 'World Bank income group': 'High-income', 'Country': 'Andorra', 'Sex': 'Female', 'Display Value': 28, 'Numeric': 28.0, 'Low': '', 'High': '', 'Comments': ''}
{'Indicator': 'Life expectancy at age 60 (years)', 'PUBLISH STATES': 'Published', 'Year': 2000, 'WHO region': 'Europe', 'World Bank income group': 'High-income'